In [3]:
import numpy as np
import sys
from numpy import array
import string
import pickle as plk
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
!pip install xlrd

    100% |████████████████████████████████| 112kB 2.6MB/s 


In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
SEQUENCE_LEN = 50
BATCH_SIZE = 512
EPOCHS = 10
HIDDEN_LAYERS_DIM = 512
LAYER_COUNT = 4
DROPOUT = 0.2

df = pd.read_excel('Q&A.xlsx',header = 0)
l = df['Breakout'].unique() # 79 unique analyst
dic = {} #create dictionary for questions
for category in l:
    list_ = list(df.loc[df['Breakout']  == category]['Question'])
    dic[category] = list_

In [5]:
df_new = pd.read_csv('combined_csv_forecast.csv',header = 0, encoding='latin-1')
l2 = df_new['sub_lvl'].unique()
#create dictionary for questions
for category in l2:
    list_ = list(df_new.loc[df_new['sub_lvl']  == category]['Question'])
    dic[category] = list_

In [6]:
#character vocabulary
UNK_IDX = 98 # add unknow as the 
characters = list(string.printable)
characters.remove('\x0b')
characters.remove('\x0c')
VOCABULARY_SIZE = len(characters)
characters_to_ix = {c:i for i,c in enumerate(characters)}
print("vocabulary len = %d" % VOCABULARY_SIZE)
characters.append('<unk>')
print(characters)

vocabulary len = 98
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' ', '\t', '\n', '\r', '<unk>']


In [7]:
dic_len = len(dic['Balance sheet']) + len(dic['FIRMWIDE-BALANCE SHEET'])
all_data = dic['Balance sheet'] + dic['FIRMWIDE-BALANCE SHEET']
train_data = all_data[:int(dic_len*0.8)]
val_data = all_data[int(dic_len*0.8): dic_len]

In [14]:
import spacy
import re
import string
# Load English tokenizer, tagger, parser, NER and word vectors
UNK_IDX = 0


#tokenize sentence by sentence
def question_split(input_):
    list_ = []
    for q in input_:
        q = q.split('\n')
        if len(q) ==1 and len(tokenize(q[0]))>= 10:
            list_.append(data_clean(q[0]))
        else:
            for i in range(len(q)):
                if len(tokenize(q[i])) >= 10:
                    list_.append(data_clean(q[i]))
    return list_

def data_clean(sent):
    
    sent = re.sub(' y ou', ' you', sent)
    sent = re.sub(' y es', ' yes', sent)
    sent = re.sub(' v o', ' vo', sent)
    sent = re.sub("don't", 'dont', sent)
    sent = re.sub( r'([a-zA-Z])([,.!?])', r'\1 \2', sent)
    return sent

def tokenize(sent):
    sent = sent.lower()
    sent = re.sub(' y ou', ' you', sent)
    sent = re.sub(' y es', ' yes', sent)
    sent = re.sub(' v o', ' vo', sent)
    sent = re.sub("don't", 'dont', sent)
    sent = re.sub( r'([a-zA-Z])([,.!?])', r'\1 \2', sent)
    #keep punctions
#     sent = re.sub('[^A-Za-z&]', ' ', sent) # replace non-letter with space
#     sent = re.sub(r'\b[a-zA-Z]\b', '', sent) #
#     sent = re.sub('^[0-9]+', '', sent)
    return sent.split() #count how many words in there

def create_letter(data):
    output = ''
    for i in range(len(data)):
        out = ''
        for l in data[i]:
            out += l
        output += out
    return output

def describe_batch(X, y, n=3):

    for i in range(n):
        out = ""
        for seq in range(SEQUENCE_LEN):
            out += characters[X[i,seq,:].argmax()]
        next_char = characters[y[i,:].argmax()]
        
        print("sample %d: ...%s ---> '%s'" % (
            i,
            out[-20:],
            next_char
        ))

def batch_generator(text, count):

    while True:
        for batch_ix in range(count):
            X = np.zeros((BATCH_SIZE, SEQUENCE_LEN, VOCABULARY_SIZE+1))
            y = np.zeros((BATCH_SIZE, VOCABULARY_SIZE+1))

            offset = BATCH_SIZE * batch_ix

            for sample_ix in range(BATCH_SIZE):
                start = offset + sample_ix
                for s in range(SEQUENCE_LEN):
                    if text[start+s] in characters_to_ix.keys():
                        X[sample_ix, s, characters_to_ix[text[start+s]]] = 1
                    else:
                        X[sample_ix, s, UNK_IDX] = 1 # adding the unkown index
                if text[sample_start+s+1] in characters_to_ix.keys():
                    y[sample_ix, characters_to_ix[text[start+s+1]]]=1
                else:
                    y[sample_ix, UNK_IDX]=1
            yield X, y


In [17]:
train_text = create_letter(question_split(train_data))
val_text = create_letter(question_split(val_data))

In [18]:
file = open('train_text.txt', 'w')
file.write(train_text)
file.close()

file = open('val_text.txt', 'w')
file.write(val_text)
file.close()

In [19]:
with open('train_text.txt', "r") as f:
    text_train = f.read()
with open('val_text.txt', "r") as f:
    text_val = f.read()

text_train_len = len(text_train)
text_val_len = len(text_val)
print("Total of %d characters" % (text_train_len + text_val_len))

for ix, (X,y) in enumerate(batch_generator(text_train, count=1)):
    # describe some samples from the first batch
    describe_batch(X, y, n=5)
    break

Total of 665287 characters
sample 0: ...u have an outlook fo ---> 'r'
sample 1: ... have an outlook for ---> ' '
sample 2: ...have an outlook for  ---> 'h'
sample 3: ...ave an outlook for h ---> 'o'
sample 4: ...ve an outlook for ho ---> 'w'


In [20]:
def build_model():
    """Build a Keras sequential model for training the char-rnn"""
    model = Sequential()
    for i in range(LAYER_COUNT):
        model.add(
            Bidirectional(LSTM(HIDDEN_LAYERS_DIM, return_sequences=True if (i!=(LAYER_COUNT-1)) else False), 
            input_shape=(SEQUENCE_LEN, VOCABULARY_SIZE+1)
            ) # add a Bidiretional layer here
        )
        model.add(Dropout(DROPOUT))
    
    model.add(Dense(VOCABULARY_SIZE+1))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

In [21]:
def build_LSTM_model():
    """Build a Keras sequential model for training the char-rnn"""
    model = Sequential()
    for i in range(LAYER_COUNT):
        model.add(
            LSTM(
                HIDDEN_LAYERS_DIM, 
                return_sequences=True if (i!=(LAYER_COUNT-1)) else False,
                input_shape=(SEQUENCE_LEN, VOCABULARY_SIZE+1),
            )
        )
        model.add(Dropout(DROPOUT))
    
    model.add(Dense(VOCABULARY_SIZE+1))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

In [22]:
training_model = build_model()
training_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [23]:
# training_model = build_model()
# training_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

train_batch_count = (text_train_len - SEQUENCE_LEN) // BATCH_SIZE
val_batch_count = (text_val_len - SEQUENCE_LEN) // BATCH_SIZE
print("training batch count: %d" % train_batch_count)
print("validation batch count: %d" % val_batch_count)

# checkpoint
from keras.callbacks import ModelCheckpoint, EarlyStopping
filepath="char_RNN_more_data.hdf5"
# checkpoint = ModelCheckpoint(filepath, save_weights_only=True)
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=0)

callbacks_list = [checkpoint, early_stopping]

training batch count: 1049
validation batch count: 249


In [0]:
history_5 = training_model.fit_generator(
    batch_generator(text_train, count=train_batch_count),
    train_batch_count,
    max_queue_size=1, # no more than one queued batch in RAM
    epochs=EPOCHS,
    callbacks=callbacks_list,
    validation_data=batch_generator(text_val, count=val_batch_count),
    validation_steps=val_batch_count,
    initial_epoch=0)

Epoch 1/5
1049/1049 [==============================] - 4300s 4s/step - loss: 2.2319 - acc: 0.3807 - val_loss: 1.4925 - val_acc: 0.5677

Epoch 00001: loss improved from inf to 2.23190, saving model to char_RNN_more_data.hdf5
Epoch 2/5
1049/1049 [==============================] - 4271s 4s/step - loss: 1.4201 - acc: 0.5880 - val_loss: 1.2251 - val_acc: 0.6385

Epoch 00002: loss improved from 2.23190 to 1.42006, saving model to char_RNN_more_data.hdf5
Epoch 3/5
1049/1049 [==============================] - 4240s 4s/step - loss: 1.2329 - acc: 0.6361 - val_loss: 1.1348 - val_acc: 0.6634

Epoch 00003: loss improved from 1.42006 to 1.23291, saving model to char_RNN_more_data.hdf5
Epoch 4/5
1049/1049 [==============================] - 4237s 4s/step - loss: 1.1425 - acc: 0.6606 - val_loss: 1.0704 - val_acc: 0.6820

Epoch 00004: loss improved from 1.23291 to 1.14254, saving model to char_RNN_more_data.hdf5
Epoch 5/5
1049/1049 [==============================] - 4236s 4s/step - loss: 1.0793 - acc: 0

In [0]:
training_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 60, 1024)          2506752   
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 1024)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 60, 1024)          6295552   
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 1024)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 60, 1024)          6295552   
_________________________________________________________________
dropout_3 (Dropout)          (None, 60, 1024)          0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 1024)              6295552   
__________

In [25]:
# training_model_2 = build_LSTM_model()
# filepath="char_LSTM_data.hdf5"
# # checkpoint = ModelCheckpoint(filepath, save_weights_only=True)
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# # early stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=0)

# callbacks_list = [checkpoint, early_stopping]

# # training_model_2.load_weights(filepath)
# history_2 = training_model_2.fit_generator(
#     batch_generator(text_train, count=train_batch_count),
#     train_batch_count,
#     max_queue_size=1, # no more than one queued batch in RAM
#     epochs=EPOCHS,
#     callbacks=callbacks_list,
#     validation_data=batch_generator(text_val, count=val_batch_count),
#     validation_steps=val_batch_count,
#     initial_epoch=0)

In [0]:
import json
# Get the dictionary containing each metric and the loss for each epoch
history_dict = history_5.history
# Save it under the form of a json file
json.dump(history_dict, open('history_5', 'w'))

In [0]:
training_model.save('char_rnn_model_new_data.h5')

In [0]:
text_train

'On net interest income , do you have an outlook for how the net interest income dollars could trend from here , assuming that you don’t get much help from higher rates , what are the key drivers ? And what’s kind of your outlook for NIM and NII dollars for the year ?On your deposit discussion about pushing out , I think you said during your Investor Day that you would like to get about $3B of non-core deposits off the balance sheet , and I know you said that you hope to make progress in Q2. How should we model that out ? And what type of benefit have you modeled out to the NIM with that $3B of deposits ?Thank you so much for giving us a detailed rundown in terms of how you’re thinking about your liquidity strategy . And I also appreciate the $28.4B in deposit growth came from consumer and escrow . But taking a step back of the $1.2 trillion in period-end deposits , how much would you classify as non-operational corporate ? And I guess , we’re just wondering , is adopting a similar str

In [0]:
!pip install tensorflowjs

    100% |████████████████████████████████| 307kB 7.9MB/s 
    100% |████████████████████████████████| 13.9MB 3.5MB/s 
keras 2.2.2 has requirement keras-applications==1.0.4, but you'll have keras-applications 1.0.6 which is incompatible.
keras 2.2.2 has requirement keras-preprocessing==1.0.2, but you'll have keras-preprocessing 1.0.5 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [0]:
!mkdir model
!tensorflowjs_converter --input_format keras char_rnn_model_new_data.h5 model/

Using TensorFlow backend.


In [0]:
!zip -r model.zip model 

  adding: model/ (stored 0%)
  adding: model/group1-shard10of21 (deflated 7%)
  adding: model/group1-shard2of21 (deflated 7%)
  adding: model/group1-shard14of21 (deflated 7%)
  adding: model/group1-shard7of21 (deflated 7%)
  adding: model/group1-shard12of21 (deflated 7%)
  adding: model/group1-shard15of21 (deflated 7%)
  adding: model/group1-shard3of21 (deflated 7%)
  adding: model/group1-shard13of21 (deflated 7%)
  adding: model/group1-shard18of21 (deflated 7%)
  adding: model/group1-shard5of21 (deflated 7%)
  adding: model/group1-shard17of21 (deflated 7%)
  adding: model/group1-shard4of21 (deflated 7%)
  adding: model/group1-shard8of21 (deflated 7%)
  adding: model/group1-shard21of21 (deflated 7%)
  adding: model/group1-shard11of21 (deflated 7%)
  adding: model/group1-shard9of21 (deflated 7%)
  adding: model/group1-shard19of21 (deflated 7%)
  adding: model/group1-shard16of21 (deflated 7%)
  adding: model/model.json (deflated 88%)
  adding: model/group1-shard1of21 (deflated 7%)
  addi

In [0]:
from google.colab import files
files.download('model.zip')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 44052, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [26]:
test_model = Sequential()
for i in range(LAYER_COUNT):
        test_model.add(Bidirectional(LSTM(HIDDEN_LAYERS_DIM, return_sequences=True if (i!=(LAYER_COUNT-1)) else False), 
            input_shape=(1, VOCABULARY_SIZE+1)) # add a Bidiretional layer here
        )
test_model.add(Dense(VOCABULARY_SIZE+1))
test_model.add(Activation('softmax'))
test_model.compile(loss='categorical_crossentropy', optimizer="adam")

In [30]:
# test_model.load_weights('char_rnn_model_new_data.h5')

In [0]:
def sample(preds, temperature=1.0):

    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def predict_next_char(model, current_char, diversity=1.0):

    x = np.zeros((1, 1, VOCABULARY_SIZE+1))
    if current_char in characters_to_ix.keys():
        x[:,:,characters_to_ix[current_char]] = 1
        y = model.predict(x, batch_size=1)
        next_char_ix = sample(y[0,:], temperature=diversity)
        next_char = characters[next_char_ix]
    return next_char



def generate_text(model, seed="I am", count=500):

    model.reset_states()
    for s in seed[:-1]:
        next_char = predict_next_char(model, s)
    current_char = seed[-1]

    sys.stdout.write("["+seed+"]")
    
    for i in range(count - len(seed)):
        next_char = predict_next_char(model, current_char, diversity=0.5)
        current_char = next_char
        sys.stdout.write(next_char)
        if next_char =='?':
            break
    print("...\n")
# def generate_text(model, seed, count=250):

#     model.reset_states()
#     sys.stdout.write("["+seed+"]")
#     for i in range(count - len(seed)):
#       next_char = predict_next_char(model, seed)
#       seed = seed[1:] + next_char
#       sys.stdout.write(next_char)
#       if next_char =='?':
#         break
#     print("...\n")

In [28]:
# text_train[:60]

In [29]:
# for i in range(5):
#     generate_text(
#         test_model,
#         seed='On net interest income , do you have an outlook for how the '
#     )